In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import japanize_matplotlib

#出力する行数を設定、省略をなくす
pd.set_option('display.max_rows', 500)

from sklearn.metrics import mean_absolute_error


In [11]:
train=pd.read_csv('train_aranged.csv')
train.head()

,ID,市区町村コード,都道府県名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引価格（総額）_log
0,40020197,40133,福岡県,谷,桜坂,7,２ＬＤＫ,3.806662,34.0,ＲＣ,住宅,住宅,第１種中高層住居専用地域,60.0,150.0,2021.50,改装済,7.041393
1,40031380,40137,福岡県,西新,西新,4,１Ｋ,2.708050,33.0,ＳＲＣ,住宅,住宅,商業地域,80.0,400.0,2020.50,未改装,6.602060
2,40030436,40137,福岡県,小田部,室見,18,４ＬＤＫ,4.382027,34.0,ＲＣ,住宅,住宅,第１種中高層住居専用地域,60.0,150.0,2020.99,改装済,7.397940
3,40015295,40132,福岡県,竹丘町,雑餉隈,5,２ＬＤＫ,4.094345,16.0,ＲＣ,住宅,住宅,商業地域,80.0,400.0,2020.25,未改装,7.278754
4,40147771,40133,福岡県,荒戸,大濠公園,6,３ＤＫ,4.094345,35.0,ＳＲＣ,住宅,NaN,商業地域,80.0,400.0,2007.50,未改装,6.929419


In [12]:
#カテゴリ変数化------------------------------------<
#市区町村コードはカテゴリ変数

use_columns=["市区町村コード", "都道府県名", "地区名", "最寄駅：名称", "間取り", "建物の構造", "用途", "今後の利用目的", "都市計画", "改装"]
for col in use_columns:
    train[col] = train[col].astype("category")

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 722564 entries, 0 to 722563
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   ID            722564 non-null  int64   
 1   市区町村コード       722564 non-null  category
 2   都道府県名         722564 non-null  category
 3   地区名           721896 non-null  category
 4   最寄駅：名称        719833 non-null  category
 5   最寄駅：距離（分）     722564 non-null  int64   
 6   間取り           696674 non-null  category
 7   面積（㎡）         722564 non-null  float64 
 8   建築年           722564 non-null  float64 
 9   建物の構造         703412 non-null  category
 10  用途            651706 non-null  category
 11  今後の利用目的       356647 non-null  category
 12  都市計画          702746 non-null  category
 13  建ぺい率（％）       698403 non-null  float64 
 14  容積率（％）        698403 non-null  float64 
 15  取引時点          722564 non-null  float64 
 16  改装            654490 non-null  category
 17  取引価格（総額）_log  722564 non-null

In [13]:
test=pd.read_csv('test_adjusted.csv')
test.head()

,ID,市区町村コード,都道府県名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装
0,1000078,1101,北海道,大通西,西１１丁目,1,３ＬＤＫ,4.317488,6.0,ＲＣ,住宅,住宅,商業地域,80.0,600.0,2021.50,未改装
1,1000121,1101,北海道,大通西,西１８丁目,0,１ＤＫ,3.401197,45.0,ＳＲＣ,住宅,住宅,商業地域,80.0,400.0,2021.50,改装済
2,1000123,1101,北海道,大通西,西１８丁目,3,３ＬＤＫ,4.248495,38.0,ＳＲＣ,住宅,住宅,商業地域,80.0,400.0,2021.50,改装済
3,1000127,1101,北海道,大通西,西１８丁目,2,２ＬＤＫ,3.912023,33.0,ＳＲＣ,住宅,住宅,商業地域,80.0,400.0,2021.99,未改装
4,1000129,1101,北海道,大通西,西１８丁目,2,１ＤＫ,3.806662,31.0,ＳＲＣ,住宅,住宅,商業地域,80.0,400.0,2021.99,改装済


In [14]:
#カテゴリ変数化------------------------------------<
use_columns=["市区町村コード", "都道府県名", "地区名", "最寄駅：名称", "間取り", "建物の構造", "用途", "今後の利用目的", "都市計画", "改装"]
for col in use_columns:
    test[col] = test[col].astype("category")

test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21005 entries, 0 to 21004
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   ID         21005 non-null  int64   
 1   市区町村コード    21005 non-null  category
 2   都道府県名      21005 non-null  category
 3   地区名        21004 non-null  category
 4   最寄駅：名称     21000 non-null  category
 5   最寄駅：距離（分）  21005 non-null  int64   
 6   間取り        19847 non-null  category
 7   面積（㎡）      21005 non-null  float64 
 8   建築年        21005 non-null  float64 
 9   建物の構造      19776 non-null  category
 10  用途         15396 non-null  category
 11  今後の利用目的    20081 non-null  category
 12  都市計画       20701 non-null  category
 13  建ぺい率（％）    20596 non-null  float64 
 14  容積率（％）     20596 non-null  float64 
 15  取引時点       21005 non-null  float64 
 16  改装         18032 non-null  category
dtypes: category(10), float64(5), int64(2)
memory usage: 1.7 MB


In [15]:
'''
１平米あたりの価格の対数をカラムとして加える
これが目的変数
'''
#面積の対数をとってカラムとする
train_area_log=train

#1平米あたりの価格の対数をとったものが、一時的なtmp目的変数
train_area_log['価格/面積_log']=train_area_log['取引価格（総額）_log']-train_area_log['面積（㎡）']
train_area_log=train_area_log.drop('ID', axis=1)

train_area_log.head(3)

,市区町村コード,都道府県名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引価格（総額）_log,価格/面積_log
0,40133,福岡県,谷,桜坂,7,２ＬＤＫ,3.806662,34.0,ＲＣ,住宅,住宅,第１種中高層住居専用地域,60.0,150.0,2021.50,改装済,7.041393,3.234730
1,40137,福岡県,西新,西新,4,１Ｋ,2.708050,33.0,ＳＲＣ,住宅,住宅,商業地域,80.0,400.0,2020.50,未改装,6.602060,3.894010
2,40137,福岡県,小田部,室見,18,４ＬＤＫ,4.382027,34.0,ＲＣ,住宅,住宅,第１種中高層住居専用地域,60.0,150.0,2020.99,改装済,7.397940,3.015913


In [16]:

"""
学習,検証9:テスト1
"""

from sklearn.model_selection import train_test_split

#説明変数 ： X
X=train_area_log.drop(['取引価格（総額）_log','価格/面積_log'], axis=1)

#目的変数 : y
y=train_area_log[['取引価格（総額）_log','価格/面積_log']]

#学習検証データとテストデータに分割, 9:1
train_val_X, test_X, train_val_y, test_y = train_test_split(X, y, test_size=0.1, random_state=1)

#学習データと検証データに分割, 8:2
train_X, val_X, train_y, val_y = train_test_split(train_val_X, train_val_y, test_size=0.2, random_state=1)

#面積あたりの価格予測したい
train_y_tmp=train_y['価格/面積_log']
val_y_tmp=val_y['価格/面積_log']
test_y_tmp=test_y['価格/面積_log']

#本来予測したい
train_y=train_y['取引価格（総額）_log']
val_y=val_y['取引価格（総額）_log']
test_y=test_y['取引価格（総額）_log']
y1=y['取引価格（総額）_log']


In [17]:
import optuna.integration.lightgbm as lgb_tune
import lightgbm as lgb
import time

# trains=lgb_tune.Dataset(train_X, label=train_y_tmp)
# valids=lgb_tune.Dataset(val_X, label=val_y_tmp)
trains=lgb.Dataset(train_X, label=train_y_tmp)
valids=lgb.Dataset(val_X, label=val_y_tmp)

# params = {
#     "objective": "regression_l1",
#     "metric": "mae",
#     'n_estimators': 10000, 
#     'verbosity':-1,
#     'n_jobs':-1
# }

# s=time.time()
# lgb_o = lgb_tune.train(params, trains, 
#                     valid_sets=[trains, valids],
#                     verbose_eval=False,
#                     early_stopping_rounds=100,
#                     )
# e=time.time()
# print(e-s)

In [ ]:
lgb_o.params

'bagging_fraction': 0.8933105005420345, 'bagging_freq': 6 
'num_leaves': 253 
'feature_fraction': 0.516 
'lambda_l1': 5.118671429810247, 'lambda_l2': 7.454192125357557e-06
'min_child_samples': 25 


In [20]:
params={
 'objective': 'regression_l1',
 'metric': 'l1',
 'verbosity': -1,
 'n_jobs': -1,
 'feature_pre_filter': False,
 'lambda_l1': 5.118671429810247,
 'lambda_l2': 7.454192125357557e-06,
 'num_leaves': 253,
 'feature_fraction': 0.516,
 'bagging_fraction': 0.8933105005420345,
 'bagging_freq': 6,
 'min_child_samples': 25,
 'num_iterations': 10000,
 'early_stopping_round': 100,
 #'categorical_column': [2, 3, 4, 6, 9, 10, 11, 12, 16],
 'learning_rate': 0.03,
 'random_state': 42
}
lgb_o = lgb.train(params, trains, valid_sets=[trains,valids], num_boost_round=10000, early_stopping_rounds=100, verbose_eval=10)

Training until validation scores don't improve for 100 rounds
[10]	training's l1: 0.302574	valid_1's l1: 0.303213
[20]	training's l1: 0.243143	valid_1's l1: 0.244362
[30]	training's l1: 0.198396	valid_1's l1: 0.200139
[40]	training's l1: 0.16851	valid_1's l1: 0.170471
[50]	training's l1: 0.145355	valid_1's l1: 0.147721
[60]	training's l1: 0.128783	valid_1's l1: 0.131476
[70]	training's l1: 0.116936	valid_1's l1: 0.120002
[80]	training's l1: 0.107292	valid_1's l1: 0.110763
[90]	training's l1: 0.100755	valid_1's l1: 0.104472
[100]	training's l1: 0.0952107	valid_1's l1: 0.0992939
[110]	training's l1: 0.0910794	valid_1's l1: 0.0955277
[120]	training's l1: 0.087545	valid_1's l1: 0.0922133
[130]	training's l1: 0.0846624	valid_1's l1: 0.0895525
[140]	training's l1: 0.0823104	valid_1's l1: 0.0874479
[150]	training's l1: 0.0808008	valid_1's l1: 0.086048
[160]	training's l1: 0.0792555	valid_1's l1: 0.0848181
[170]	training's l1: 0.0779046	valid_1's l1: 0.0837402
[180]	training's l1: 0.076606	val

[1500]	training's l1: 0.0594813	valid_1's l1: 0.0728833
[1510]	training's l1: 0.0594607	valid_1's l1: 0.0728774
[1520]	training's l1: 0.0594375	valid_1's l1: 0.0728674
[1530]	training's l1: 0.0594197	valid_1's l1: 0.0728633
[1540]	training's l1: 0.0594012	valid_1's l1: 0.0728576
[1550]	training's l1: 0.0593813	valid_1's l1: 0.0728518
[1560]	training's l1: 0.0593588	valid_1's l1: 0.0728425
[1570]	training's l1: 0.0593405	valid_1's l1: 0.0728374
[1580]	training's l1: 0.0593219	valid_1's l1: 0.0728327
[1590]	training's l1: 0.0593028	valid_1's l1: 0.0728259
[1600]	training's l1: 0.0592849	valid_1's l1: 0.0728203
[1610]	training's l1: 0.0592665	valid_1's l1: 0.0728153
[1620]	training's l1: 0.0592499	valid_1's l1: 0.0728107
[1630]	training's l1: 0.0592305	valid_1's l1: 0.0728047
[1640]	training's l1: 0.0592117	valid_1's l1: 0.0727983
[1650]	training's l1: 0.0591899	valid_1's l1: 0.0727917
[1660]	training's l1: 0.0591666	valid_1's l1: 0.0727862
[1670]	training's l1: 0.0591498	valid_1's l1: 0.

[2970]	training's l1: 0.057384	valid_1's l1: 0.0723019
[2980]	training's l1: 0.0573744	valid_1's l1: 0.0723004
[2990]	training's l1: 0.0573643	valid_1's l1: 0.0722984
[3000]	training's l1: 0.0573547	valid_1's l1: 0.0722969
[3010]	training's l1: 0.0573417	valid_1's l1: 0.0722929
[3020]	training's l1: 0.0573318	valid_1's l1: 0.07229
[3030]	training's l1: 0.0573216	valid_1's l1: 0.0722863
[3040]	training's l1: 0.057311	valid_1's l1: 0.0722835
[3050]	training's l1: 0.0573013	valid_1's l1: 0.0722804
[3060]	training's l1: 0.0572904	valid_1's l1: 0.0722786
[3070]	training's l1: 0.0572799	valid_1's l1: 0.0722767
[3080]	training's l1: 0.0572698	valid_1's l1: 0.0722746
[3090]	training's l1: 0.0572589	valid_1's l1: 0.0722722
[3100]	training's l1: 0.0572475	valid_1's l1: 0.0722688
[3110]	training's l1: 0.0572404	valid_1's l1: 0.0722671
[3120]	training's l1: 0.0572307	valid_1's l1: 0.0722645
[3130]	training's l1: 0.0572188	valid_1's l1: 0.0722622
[3140]	training's l1: 0.0572107	valid_1's l1: 0.0722

[4440]	training's l1: 0.0561177	valid_1's l1: 0.0720395
[4450]	training's l1: 0.0561127	valid_1's l1: 0.0720386
[4460]	training's l1: 0.0561053	valid_1's l1: 0.072037
[4470]	training's l1: 0.0560993	valid_1's l1: 0.0720355
[4480]	training's l1: 0.0560928	valid_1's l1: 0.0720337
[4490]	training's l1: 0.0560854	valid_1's l1: 0.0720328
[4500]	training's l1: 0.0560791	valid_1's l1: 0.0720318
[4510]	training's l1: 0.0560734	valid_1's l1: 0.0720304
[4520]	training's l1: 0.0560679	valid_1's l1: 0.0720296
[4530]	training's l1: 0.0560596	valid_1's l1: 0.0720275
[4540]	training's l1: 0.0560529	valid_1's l1: 0.0720264
[4550]	training's l1: 0.0560458	valid_1's l1: 0.0720255
[4560]	training's l1: 0.0560391	valid_1's l1: 0.0720245
[4570]	training's l1: 0.0560323	valid_1's l1: 0.0720237
[4580]	training's l1: 0.0560243	valid_1's l1: 0.0720222
[4590]	training's l1: 0.0560152	valid_1's l1: 0.0720213
[4600]	training's l1: 0.0560085	valid_1's l1: 0.0720203
[4610]	training's l1: 0.0560016	valid_1's l1: 0.0

[5910]	training's l1: 0.0552011	valid_1's l1: 0.071883
[5920]	training's l1: 0.0551946	valid_1's l1: 0.0718821
[5930]	training's l1: 0.0551884	valid_1's l1: 0.0718812
[5940]	training's l1: 0.0551826	valid_1's l1: 0.0718803
[5950]	training's l1: 0.0551776	valid_1's l1: 0.0718799
[5960]	training's l1: 0.0551716	valid_1's l1: 0.0718787
[5970]	training's l1: 0.0551665	valid_1's l1: 0.071878
[5980]	training's l1: 0.055162	valid_1's l1: 0.0718775
[5990]	training's l1: 0.0551569	valid_1's l1: 0.0718764
[6000]	training's l1: 0.0551513	valid_1's l1: 0.0718752
[6010]	training's l1: 0.0551466	valid_1's l1: 0.0718746
[6020]	training's l1: 0.0551404	valid_1's l1: 0.0718733
[6030]	training's l1: 0.0551348	valid_1's l1: 0.0718728
[6040]	training's l1: 0.055128	valid_1's l1: 0.0718716
[6050]	training's l1: 0.0551213	valid_1's l1: 0.0718708
[6060]	training's l1: 0.055116	valid_1's l1: 0.0718701
[6070]	training's l1: 0.0551105	valid_1's l1: 0.0718689
[6080]	training's l1: 0.0551041	valid_1's l1: 0.07186

[7380]	training's l1: 0.054463	valid_1's l1: 0.0717649
[7390]	training's l1: 0.0544579	valid_1's l1: 0.0717638
[7400]	training's l1: 0.0544532	valid_1's l1: 0.0717634
[7410]	training's l1: 0.0544488	valid_1's l1: 0.0717631
[7420]	training's l1: 0.0544439	valid_1's l1: 0.0717623
[7430]	training's l1: 0.0544397	valid_1's l1: 0.0717619
[7440]	training's l1: 0.0544348	valid_1's l1: 0.0717614
[7450]	training's l1: 0.0544298	valid_1's l1: 0.0717603
[7460]	training's l1: 0.0544241	valid_1's l1: 0.0717592
[7470]	training's l1: 0.0544197	valid_1's l1: 0.0717582
[7480]	training's l1: 0.0544128	valid_1's l1: 0.0717565
[7490]	training's l1: 0.0544093	valid_1's l1: 0.0717558
[7500]	training's l1: 0.0544036	valid_1's l1: 0.0717546
[7510]	training's l1: 0.0543984	valid_1's l1: 0.0717539
[7520]	training's l1: 0.0543942	valid_1's l1: 0.0717533
[7530]	training's l1: 0.0543908	valid_1's l1: 0.071753
[7540]	training's l1: 0.0543859	valid_1's l1: 0.071752
[7550]	training's l1: 0.0543811	valid_1's l1: 0.071

[8860]	training's l1: 0.0538457	valid_1's l1: 0.0716762
[8870]	training's l1: 0.053842	valid_1's l1: 0.0716756
[8880]	training's l1: 0.0538388	valid_1's l1: 0.0716753
[8890]	training's l1: 0.0538351	valid_1's l1: 0.0716748
[8900]	training's l1: 0.0538309	valid_1's l1: 0.0716738
[8910]	training's l1: 0.053827	valid_1's l1: 0.0716736
[8920]	training's l1: 0.0538229	valid_1's l1: 0.0716732
[8930]	training's l1: 0.0538184	valid_1's l1: 0.0716723
[8940]	training's l1: 0.0538143	valid_1's l1: 0.0716715
[8950]	training's l1: 0.0538111	valid_1's l1: 0.071671
[8960]	training's l1: 0.0538071	valid_1's l1: 0.0716704
[8970]	training's l1: 0.0538037	valid_1's l1: 0.0716698
[8980]	training's l1: 0.0537992	valid_1's l1: 0.0716691
[8990]	training's l1: 0.0537953	valid_1's l1: 0.0716682
[9000]	training's l1: 0.0537909	valid_1's l1: 0.0716678
[9010]	training's l1: 0.0537871	valid_1's l1: 0.0716674
[9020]	training's l1: 0.0537829	valid_1's l1: 0.071667
[9030]	training's l1: 0.0537793	valid_1's l1: 0.0716

In [ ]:
train_pred=lgb_o.predict(train_X)+train_X['面積（㎡）']
valid_pred=lgb_o.predict(val_X)+val_X['面積（㎡）']
test_pred=lgb_o.predict(test_X)+test_X['面積（㎡）']
total_pred=lgb_o.predict(X)+X['面積（㎡）']

In [ ]:
print('学習用：',mean_absolute_error(train_pred, train_y))
print('検証用：',mean_absolute_error(valid_pred, valid_y))
print('テスト用:',mean_absolute_error(test_pred, test_y))
print('全体：',mean_absolute_error(total_pred, y1))

In [ ]:
#テスト用データで予測
submission_test_pred=lgb_o.predict(test)+test['面積（㎡）']

#提出ファイル
submission_test_pred.to_csv('submission_lgb_o1.csv', header=None, index=None)